# <span style="font-width:bold; font-size: 3rem; color:#1EB182;">**Hopsworks Feature Store** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Model training & UI Exploration</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/eletricity/4_model_training_and_registration.ipynb)


In this last notebook, you will train a model on the dataset we created in the previous tutorial. You will train the model using standard Python and Scikit-learn, although it could just as well be trained with other machine learning frameworks such as PySpark, TensorFlow, and PyTorch. In the end some exploration that can be done in Hopsworks will be shown, notably the search functions and the lineage.

## 🗒️ This notebook is divided into 3 main sections:
1. **Loading the training data**
2. **Train the model**
3. **Register model to Hopsworks model registry**.

![tutorial-flow](../../images/03_model.png)

### <span style="color:#ff5f27;">📝 Importing Libraries</span>

In [ ]:
!pip install -U hopsworks --quiet
!pip install -U tensorflow --quiet

In [ ]:
from __future__ import print_function

import tensorflow as tf

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

%config InlineBackend.figure_format='retina'
%matplotlib inline

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina', quality=100)

import warnings
warnings.filterwarnings('ignore')

---

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

---

## <span style="color:#ff5f27;">🪝 Feature View and Training Dataset Retrieval</span>

In [ ]:
feature_view = fs.get_feature_view(
    name = 'electricity_feature_view',
    version = 1
)

In [ ]:
X_train, _ = feature_view.get_training_data(1)
X_val, _ = feature_view.get_training_data(2)
X_test, _ = feature_view.get_training_data(3)

In [ ]:
X_train.sort_values(["timestamp"], inplace=True)
X_val.sort_values(["timestamp"], inplace=True)
X_test.sort_values(["timestamp"], inplace=True)

In [ ]:
y_train = X_train[["price_se1", "price_se2", "price_se3", "price_se4"]]
y_val = X_val[["price_se1", "price_se2", "price_se3", "price_se4"]]
y_test = X_test[["price_se1", "price_se2", "price_se3", "price_se4"]]

In [ ]:
X_train.drop(["day", "timestamp"], axis = 1, inplace = True)
X_val.drop(["day", "timestamp"], axis = 1, inplace = True)
X_test.drop(["day", "timestamp"], axis = 1, inplace = True)

In [ ]:
X_test

---

## <span style="color:#ff5f27;">🗃 Window timeseries dataset </span>


In [ ]:
import tensorflow as tf
# https://www.tensorflow.org/tutorials/structured_data/time_series
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
               df_train, val_df, test_df,
               label_columns=None, batch_size=32):
        # Store the raw data.
        self.df_train = df_train
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
          self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                               enumerate(df_train.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

        self.batch_size = batch_size
    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

    def split_window(self, features):
      inputs = features[:, self.input_slice, :]
      labels = features[:, self.labels_slice, :]
      if self.label_columns is not None:
        labels = tf.stack(
            [labels[:, :, self.column_indices[name]] for name in self.label_columns],
            axis=-1)

      # Slicing doesn't preserve static shape information, so set the shapes
      # manually. This way the `tf.data.Datasets` are easier to inspect.
      inputs.set_shape([None, self.input_width, None])
      labels.set_shape([None, self.label_width, None])

      return inputs, labels

    def plot(self, plot_col, model=None, max_subplots=3):
      inputs, labels = self.example
      plt.figure(figsize=(12, 8))
      plot_col_index = self.column_indices[plot_col]
      max_n = min(max_subplots, len(inputs))
      for n in range(max_n):
        plt.subplot(max_n, 1, n+1)
        plt.ylabel(f'{plot_col} [normed]')
        plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                 label='Inputs', marker='.', zorder=-10)

        if self.label_columns:
          label_col_index = self.label_columns_indices.get(plot_col, None)
        else:
          label_col_index = plot_col_index

        if label_col_index is None:
          continue

        plt.scatter(self.label_indices, labels[n, :, label_col_index],
                    edgecolors='k', label='Labels', c='#2ca02c', s=64)
        if model is not None:
          predictions = model(inputs)
          plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                      marker='X', edgecolors='k', label='Predictions',
                      c='#ff7f0e', s=64)

        if n == 0:
          plt.legend()

      plt.xlabel('Time [h]')

    #make_dataset method will take a time series DataFrame and convert it to a tf.data.Dataset of (input_window, label_window) 
    # pairs using the tf.keras.utils.timeseries_dataset_from_array function:
    def make_dataset(self, data):
      data = np.array(data, dtype=np.float32)
      ds = tf.keras.utils.timeseries_dataset_from_array(
          data=data,
          targets=None,
          sequence_length=self.total_window_size,
          sequence_stride=1,
          shuffle=False,
          batch_size=self.batch_size,)    
      ds = ds.map(self.split_window)
      ds = ds.repeat(1000)
      ds = ds.prefetch(10)  
      return ds

    @property
    def train(self):
      return self.make_dataset(self.df_train)

    @property
    def val(self):
      return self.make_dataset(self.val_df)

    @property
    def test(self):
      return self.make_dataset(self.test_df)

    @property
    def example(self):
      """Get and cache an example batch of `inputs, labels` for plotting."""
      result = getattr(self, '_example', None)
      if result is None:
        # No example batch was found, so get one from the `.train` dataset
        result = next(iter(self.test))
        # And cache it for next time
        self._example = result
      return result

In [ ]:
n_step_window = WindowGenerator(df_train=X_train, val_df=X_val, test_df=X_test, input_width=4, label_width=4, shift=4, label_columns=["price_se1", "price_se2", "price_se3", "price_se4"])
n_step_window

In [ ]:
inputs, labels = n_step_window.example
print(inputs.shape)
print(labels.shape)
print(n_step_window.label_indices)

In [ ]:
X = np.arange(100)
Y = X * 2
X = pd.DataFrame(X, columns = ["X"])
X["Y"] = Y
n_step_window = WindowGenerator(df_train=X, val_df=X, test_df=X, input_width=4, label_width=4, shift=4, label_columns=["Y"], batch_size=4)
a = [i[0].numpy() for i in n_step_window.val]
b = [i[1].numpy() for i in n_step_window.val]
a[0][0]
b[0][0]
n_step_window.plot(plot_col="Y", max_subplots=2)

In [ ]:
for example_inputs, example_labels in n_step_window.train.take(1):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')

---

## <span style="color:#ff5f27;">🧬 Modeling</span>

In [ ]:
def build_model(input_dim):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv1D(filters = 64, kernel_size=1, padding='same', kernel_initializer="uniform", input_shape=(input_dim[0], input_dim[1])))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))        

    model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size= 1,padding='same',  kernel_initializer="uniform"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))       

    model.add(tf.keras.layers.Conv1D(filters = 16, kernel_size= 1,padding='same',  kernel_initializer="uniform"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))   
    model.add(tf.keras.layers.MaxPooling1D(pool_size=1, padding='same'))       
    
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=16, return_sequences=True))) 
    model.add(tf.keras.layers.Dropout(rate=0.1))
    model.add(tf.keras.layers.Dense(units=4))
    
    model.summary()
    model.compile(loss='mae', optimizer='adam')
    return model

In [ ]:
model = build_model([4, 29])

In [ ]:
from timeit import default_timer as timer
start = timer()

history = model.fit(n_step_window.train,
                    epochs=50,
                    verbose=0,
                    steps_per_epoch=200,
                    validation_data=n_step_window.train,
                    validation_steps=1,                    
                   )
end = timer()
print(end - start)

In [ ]:
inputs, labels = n_step_window.example
prediction_test = model.predict(inputs)
print(prediction_test.shape)
print(labels.shape)

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt


loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

loss_values50 = loss_values
val_loss_values50 = val_loss_values
epochs = range(1, len(loss_values50) + 1)
plt.plot(epochs, loss_values50, 'b',color = 'blue', label='Training loss')
plt.plot(epochs, val_loss_values50, 'b',color='red', label='Validation loss')
plt.rc('font', size = 18)
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.xticks(epochs)
fig = plt.gcf()
fig.set_size_inches(15,7)
plt.show()

In [ ]:
n_step_window.plot(plot_col="price_se4", max_subplots=3, model=model.predict)

In [ ]:
se1_actual = []
se2_actual = []
se3_actual = []
se4_actual = []

inputs, labels = n_step_window.example
for batch_n in range(len(labels)):
    batch = labels[batch_n]
    for window_n in range(4):
        se1_actual.append(batch[window_n][0].numpy())
        se2_actual.append(batch[window_n][1].numpy())
        se3_actual.append(batch[window_n][2].numpy())
        se4_actual.append(batch[window_n][3].numpy())

In [ ]:
se1_pred = []
se2_pred = []
se3_pred = []
se4_pred = []

prediction_test = model.predict(inputs)
for batch_n in range(len(prediction_test)):
    batch = prediction_test[batch_n]
    for window_n in range(4):
        se1_pred.append(batch[window_n][0])
        se2_pred.append(batch[window_n][1])
        se3_pred.append(batch[window_n][2])
        se4_pred.append(batch[window_n][3])

In [ ]:
y_test.head(10)

In [ ]:
se3_actual[1]

In [ ]:
plt.plot(se1_pred,color='red', label='test SE1 price prediction')
plt.plot(se1_actual, color='blue', label='test actual')
plt.xlabel('Time')
plt.ylabel('Price (scaled)')
plt.legend(loc='upper left')
fig = plt.gcf()
fig.set_size_inches(15, 5)
plt.show()

In [ ]:
plt.plot(se2_pred,color='red', label='test SE2 price prediction')
plt.plot(se2_actual, color='blue', label='test actual')
plt.xlabel('Time')
plt.ylabel('Price (scaled)')
plt.legend(loc='upper left')
fig = plt.gcf()
fig.set_size_inches(15, 5)
plt.show()

In [ ]:
plt.plot(se3_pred,color='red', label='test SE3 price prediction')
plt.plot(se3_actual, color='blue', label='test actual')
plt.xlabel('Time')
plt.ylabel('Price (scaled)')
plt.legend(loc='upper left')
fig = plt.gcf()
fig.set_size_inches(15, 5)
plt.show()

In [ ]:
plt.plot(se4_pred,color='red', label='test SE4 price prediction')
plt.plot(se4_actual, color='blue', label='test actual')
plt.xlabel('Time')
plt.ylabel('Price (scaled)')
plt.legend(loc='upper left')
fig = plt.gcf()
fig.set_size_inches(15, 5)
plt.show()

---

## <span style='color:#ff5f27'>🗄 Model Registry</span>

One of the features in Hopsworks is the model registry. This is where you can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

In [ ]:
export_path = "electricity_price_model"
print('Exporting trained model to: {}'.format(export_path))
tf.saved_model.save(model, export_path) 

In [ ]:
mr = project.get_model_registry()
metrics={'loss': history_dict['val_loss'][0]} 

mr_model = mr.tensorflow.create_model(
    name="electricity_price_prediction_model",
    metrics=metrics,
    description="Daily electricity price prediction model.",
    input_example=n_step_window.example[0].numpy()
)

In [ ]:
mr_model.save(export_path)